In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [9]:
import shutil
import yaml
import csv
from pyoscomp.scenario.manager import ScenarioManager

# --- Helper to clean up previous runs for this demo ---
WORK_DIR = "demo_output"
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)
os.makedirs(WORK_DIR)

# Create a minimal config file for the demo
config_content = """
REGION:
  type: set
YEAR:
  type: set
SEASON:
  type: set
DAYTYPE:
  type: set
DAILYTIMEBRACKET:
  type: set
TIMESLICE:
  type: set
YearSplit:
  type: param
  columns: ["TIMESLICE", "YEAR", "VALUE"]
DaySplit:
  type: param
  columns: ["DAILYTIMEBRACKET", "YEAR", "VALUE"]
DaysInDayType:
  type: param
  columns: ["SEASON", "DAYTYPE", "YEAR", "VALUE"]
"""

config_path = os.path.join(WORK_DIR, "osemosys_config.yaml")
with open(config_path, "w") as f:
    f.write(config_content)

# 1. Create Scenario
manager = ScenarioManager(
    tag="Scenario_Uniform", 
    parent_dir=WORK_DIR,
    config_path=config_path,
    master_path=os.path.join(WORK_DIR, "scenarios_master_list.csv")
)
scen = manager.create_scenario()

# 2. Setup Nodes
scen.set_topology_structure(["UK"])

# 3. Setup Time (Crucial for demand profiles)
# 4 Seasons, 2 DayTypes (Weekday/Weekend), 2 Brackets (Day/Night)
scen.set_time_structure(
    years=(2020, 2050, 5),
    seasons={"Spring": 91, "Summer": 92, "Autumn": 91, "Winter": 91},
    daytypes={"Weekday": 5, "Weekend": 2},
    brackets={"Day": 16, "Night": 8}
)

# 4. Add Electricity Demand (Rising linearly)
# Profile: 'weighted'
# Logic: Winter is 2x Summer. Days are 2x Night.
scen.add_demand(
    region="UK",
    fuel="ELEC",
    trajectory={2020: 300, 2050: 600}, # TWh
    profile_type="weighted",
    weights_seasons={"Winter": 2.0, "Summer": 0.8, "Spring": 1.0, "Autumn": 1.0},
    weights_daytypes={"Weekday": 1.2, "Weekend": 0.9},
    weights_brackets={"Day": 2.0, "Night": 0.5}
)

# 5. Add Industrial Heat Demand (Flat profile)
# Logic: Factories run 24/7 constant load
scen.add_demand(
    region="UK",
    fuel="HEAT_HIGH_TEMP",
    trajectory={2020: 100, 2030: 80, 2050: 50}, # Declining
    profile_type="flat"
)

# 6. Add Flexible Demand (e.g. Hydrogen Production buffer)
scen.add_flexible_demand("UK", "H2_BUFFER", 2030, 20)

# 7. Generate CSVs
scen.build()

Scenario 'Scenario_Uniform' created with UUID: 069417df-7226-7d75-8000-18717faf6f23
Directory: demo_output/Scenario_Uniform


TypeError: Scenario.add_demand() got an unexpected keyword argument 'region'